# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os
import sys

sys.path.append('..')
from utils import *
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = "Huntington_s-Disease"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Huntington_s-Disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE135589', 'GSE147002', 'GSE147003', 'GSE147004', 'GSE154141']

Repeat the below steps for all the accession numbers

In [63]:
cohort = accession_num = "GSE154141"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Huntington_s-Disease\\GSE154141\\GSE154141_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Huntington_s-Disease\\GSE154141\\GSE154141-GPL1261_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [64]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Cell-intrinsic glial pathology is conserved across human and murine models of Huntington Disease"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [65]:
clinical_data

,!Sample_geo_accession,GSM4664357,GSM4664358,GSM4664359,GSM4664360,GSM4664361,GSM4664362,GSM4664363,GSM4664364,GSM4664365,...,GSM4664458,GSM4664459,GSM4664460,GSM4664461,GSM4664462,GSM4664463,GSM4664464,GSM4664465,GSM4664466,GSM4664467
0,!Sample_characteristics_ch1,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,...,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum,tissue: Striatum
1,!Sample_characteristics_ch1,genotype: WT,genotype: WT,genotype: WT,genotype: WT,genotype: WT,genotype: WT,genotype: WT,genotype: WT,genotype: WT,...,genotype: WT,genotype: WT,genotype: Q175,genotype: Q175,genotype: Q175,genotype: Q175,genotype: WT,genotype: WT,genotype: WT,genotype: WT
2,!Sample_characteristics_ch1,age: 6wk,age: 6wk,age: 6wk,age: 6wk,age: 6wk,age: 6wk,age: 6wk,age: 6wk,age: 6wk,...,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo,age: 12mo
3,!Sample_characteristics_ch1,cell type: astrocytes,cell type: microglia,cell type: negative cells,cell type: astrocytes,cell type: microglia,cell type: negative cells,cell type: astrocytes,cell type: microglia,cell type: negative cells,...,cell type: microglia,cell type: microglia,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes,cell type: astrocytes
4,!Sample_characteristics_ch1,facs markers: GLT1+/CD11b-,facs markers: GLT1-/CD11b+,facs markers: GLT1-/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1-/CD11b+,facs markers: GLT1-/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1-/CD11b+,facs markers: GLT1-/CD11b-,...,facs markers: GLT1-/CD11b+,facs markers: GLT1-/CD11b+,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-,facs markers: GLT1+/CD11b-


In [66]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: Striatum'],
 1: ['genotype: WT', 'genotype: R62', 'genotype: Q175'],
 2: ['age: 6wk', 'age: 12wk', 'age: 6mo', 'age: 12mo'],
 3: ['cell type: astrocytes',
  'cell type: microglia',
  'cell type: negative cells'],
 4: ['facs markers: GLT1+/CD11b-',
  'facs markers: GLT1-/CD11b+',
  'facs markers: GLT1-/CD11b-']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [67]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Huntington_s-Disease\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Huntington_s-Disease\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [68]:
is_gene_availabe = True
trait_row = 1
age_row = 2
gender_row = None

trait_type = "binary"

In [30]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [80]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'genotype huntington: Transgenic':
#         return 1  # Epilepsy present
#     else:
#         return 0  # Epilepsy not present

# def convert_age(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age (years): '):
#         try:
#             return float(age_string.split('age (years): ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: female':
#         return 0
#     elif gender_string == 'Sex: male':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender
def convert_trait(value):
    if value in ['genotype: WT', 'genotype: R62', 'genotype: Q175']:  # Assuming these are the possible values
        return 'Huntington_s-Disease'  # or 1 for affected, 0 for unaffected
    else:
        return None  # Unknown or non-standard values


def convert_age(value):
    try:
        # Convert weeks and months to years
        age_str = value.split(': ')[1].replace('wk', '*7/365').replace('mo', '*30/365')
        return eval(age_str)  # Evaluate the expression to get a numerical result
    except (IndexError, ValueError):
        return None  # Unknown or non-standard values

# def convert_age(value):
#     try:
#         return float(value.split(': ')[1])  # Extract the age value
#     except (IndexError, ValueError):
#         return None  # Unknown or non-standard values


def convert_gender(value):
    if 'f' in value.lower() or 'female' in value.lower():
        return 'female'  # or 1 if using binary representation
    elif 'm'  in value.lower() or 'male' in value.lower():
        return 'male'  # or 0 if using binary representation
    else:
        return value
        #return None  # Unknown or non-standard values



In [81]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM4664357,GSM4664358,GSM4664359,GSM4664360,GSM4664361,GSM4664362,GSM4664363,GSM4664364,GSM4664365,GSM4664366,...,GSM4664458,GSM4664459,GSM4664460,GSM4664461,GSM4664462,GSM4664463,GSM4664464,GSM4664465,GSM4664466,GSM4664467
Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,...,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease,Huntington_s-Disease
Age,0.115068,0.115068,0.115068,0.115068,0.115068,0.115068,0.115068,0.115068,0.115068,0.115068,...,0.986301,0.986301,0.986301,0.986301,0.986301,0.986301,0.986301,0.986301,0.986301,0.986301


### Genetic data preprocessing and final filtering

In [82]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4664357,GSM4664358,GSM4664359,GSM4664360,GSM4664361,GSM4664362,GSM4664363,GSM4664364,GSM4664365,GSM4664366,...,GSM4664458,GSM4664459,GSM4664460,GSM4664461,GSM4664462,GSM4664463,GSM4664464,GSM4664465,GSM4664466,GSM4664467
ID,,,,,,,,,,,,,,,,,,,,,
1415670_at,9.403976,9.974496,9.090413,9.352728,9.792785,9.154687,9.523642,10.052916,9.142804,9.218550,...,9.767204,9.817243,9.245465,9.554471,9.196820,9.453101,9.148165,9.079242,9.447168,9.077879
1415671_at,11.639311,11.214548,11.277509,11.595199,11.132460,11.110191,11.756730,11.156990,11.367047,11.632720,...,11.512597,11.735316,11.531482,11.745154,11.564668,11.838715,11.526208,11.595918,12.010248,11.633129
1415672_at,11.489716,10.620871,11.712316,11.447164,10.881411,11.665235,11.275899,10.553762,11.932644,11.560173,...,10.657727,10.892607,11.470068,11.695225,11.714739,11.654303,11.238255,11.537276,11.468672,11.601307
1415673_at,11.057820,8.627847,10.114385,10.926201,8.507834,9.940992,11.188327,8.352834,10.284325,11.029858,...,8.840496,8.692698,10.917351,10.851931,10.549544,10.714997,10.810255,10.756637,10.904644,10.870442
1415674_a_at,9.889195,9.443009,10.184199,10.106296,9.801359,10.107430,10.347441,9.871478,10.234875,9.969636,...,10.212039,10.030437,10.272982,10.362000,10.144273,10.057729,10.206751,10.216057,10.071237,10.290971


In [83]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1415670_at',
 '1415671_at',
 '1415672_at',
 '1415673_at',
 '1415674_a_at',
 '1415675_at',
 '1415676_a_at',
 '1415677_at',
 '1415678_at',
 '1415679_at',
 '1415680_at',
 '1415681_at',
 '1415682_at',
 '1415683_at',
 '1415684_at',
 '1415685_at',
 '1415686_at',
 '1415687_a_at',
 '1415688_at',
 '1415689_s_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [84]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at', '1415675_at', '1415676_a_at', '1415677_at', '1415678_at', '1415679_at', '1415680_at', '1415681_at', '1415682_at', '1415683_at', '1415684_at', '1415685_at', '1415686_at', '1415687_a_at', '1415688_at', '1415689_s_at']\n"

If not required, jump directly to the gene normalization step

In [85]:
requires_gene_mapping = True

In [86]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'Target Description': ['gb:BC024686.1 /DB_XREF=gi:19354080 /FEA=FLmRNA /CNT=416 /TID=Mm.26422.1 /TIER=FL+Stack /STK=110 /UG=Mm.26422 /LL=54161 /UG_GENE=Copg1 /DEF=Mus musculus, coatomer protein complex, subunit gamma 1, clone MGC:30335 IMAGE:3992144, mRNA, complete cds. /PROD=coatomer protein complex, subunit gamma 1 /FL=gb:AF187079.1 gb:BC024686.1 gb:NM_017477.1 gb:BC024896.1', 'gb:NM_013477

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [88]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenB

In [93]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [94]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [95]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [96]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 111 samples.


In [97]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Huntington_s-Disease', the least common label is 'Huntington_s-Disease' with 111 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Huntington_s-Disease' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 0.11506849315068493
  50% (Median): 0.23013698630136986
  75%: 0.4931506849315068
Min: 0.11506849315068493
Max: 0.9863013698630136
The distribution of the feature 'Age' in this dataset is fine.



True

In [98]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [99]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [42]:
from utils import *

In [43]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE147003'

In [44]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE147003,True,True,False,True,True,168,


In [45]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [46]:
merged_data.head()

,Huntington_s-Disease,Age,Gender,A1BG,A1CF,AACS,AADAT,AAK1,AATF,ABCA1,...,ZRANB2-AS1,ZSCAN29,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM8,ZXDC,ZYG11B,ZZEF1,ZZZ3
0,0.0,4.25,0.0,0.491806,0.899960,0.390774,0.463467,0.509208,0.917367,0.567127,...,0.602005,0.553851,0.651386,0.852842,0.804149,0.871325,0.166445,0.969788,0.940996,0.955394
1,1.0,4.25,0.0,0.490892,0.926637,0.408045,0.449019,0.538818,0.917649,0.566033,...,0.600293,0.561052,0.668281,0.852690,0.807688,0.878014,0.182718,0.972242,0.959748,0.956932
2,0.0,4.00,0.0,0.488184,0.886400,0.413423,0.461858,0.505386,0.913792,0.570111,...,0.598689,0.614170,0.663705,0.870609,0.806921,0.872751,0.152436,0.969765,0.939102,0.955801
3,1.0,4.25,1.0,0.499392,0.899555,0.390452,0.492872,0.513660,0.923118,0.572922,...,0.600632,0.336855,0.657022,0.869947,0.800576,0.871670,0.136181,0.971740,0.940295,0.951921
4,0.0,4.25,0.0,0.489025,0.903611,0.398884,0.456250,0.494501,0.927517,0.573857,...,0.593588,0.458641,0.658684,0.850950,0.806299,0.884765,0.147802,0.972008,0.949340,0.955066


In [47]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [48]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [49]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [50]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [51]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 51.52% ± 5.75%


In [52]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 62.42% ± 6.80%


In [53]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [54]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0085
Found 125 genes with non-zero coefficients associated with the trait 'Huntington_s-Disease' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
LINC01432    -5.859059              5.859059
     NEXN    -4.003604              4.003604
   TRIM39     3.943742              3.943742
    MSRB2    -3.918954              3.918954
  MIR5093     3.482877              3.482877
   TM7SF3    -3.211433              3.211433
    POLG2     3.176632              3.176632
    RIMS3    -2.977018              2.977018
LINC00210     2.432505              2.432505
  TSPAN15    -2.300655              2.300655
     ADAR    -2.224646              2.224646
     GOT1    -2.168314              2.168314
     PTH2    -2.162107              2.162107
     EDN1     2.155098              2.155098
     PRCC    -2.148877              2.148877

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [55]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0085
Found 2 significant genes associated with the trait 'Huntington_s-Disease' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
     HTT     0.473079 6.638323e-79       4.479540e-75
    FLT1    -0.245933 5.910260e-12       1.994122e-08


/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
